In [6]:
import os
import json
import random
import pandas as pd
from together import Together

Khởi tạo API và tạo prompt cho LLM để phân tích ngữ cảnh gay cười

In [7]:
# Khởi tạo client của Together API
client = Together(api_key="")

# Đường dẫn thư mục
base_dir = "D:\\DS310-P11-Nhom15"
conversation_dir = os.path.join(base_dir, "data", "Videos_text_represenations")
label_dir = os.path.join(base_dir, "data", "Humor_Explainations")

# Hàm load các đoạn hội thoại
def load_conversations_and_labels():
    data = []
    for i in range(41, 114):
        # Đọc file JSON
        conversation_file = os.path.join(conversation_dir, f"output_{i}.json")
        with open(conversation_file, "r", encoding="utf-8") as f:
            conversation_segments = json.load(f)
        
        # Đọc file Markdown
        label_file = os.path.join(label_dir, f"label-{i}.md")
        with open(label_file, "r", encoding="utf-8") as f:
            label = f.read().strip()
        
        data.append({
            "conversation_id": i,
            "conversation_segments": conversation_segments,
            "label": label
        })
    return data

# Hàm tạo prompt
def create_prompt(conversation, example_1, example_2):
    example_prompt = f"""
Bạn là chuyên gia phân tích và giải thích các tình huống hài hước trong các hội thoại. Tôi sẽ cung cấp các đoạn hội thoại, mỗi đoạn hội thoại được chia thành các segment, mỗi segment bao gồm:
* Utterance: Nội dung lời nói.
* Acoustic Features: Thông tin về giọng nói, tốc độ nói, và cách phát âm.
* Visual Description: Mô tả hình ảnh hoặc ngữ cảnh xuất hiện trong cảnh quay.
Hãy đọc các đoạn hội thoại và giải thích lý do hài hước trong các đoạn hội thoại này trong một đoạn văn ngắn, không vượt quá trong 100 từ bằng tiếng Việt.

### Ví dụ 1:
Hội thoại:
{json.dumps(example_1['conversation_segments'], ensure_ascii=False, indent=2)}
Lý do hài hước: {example_1['label']}

### Ví dụ 2:
Hội thoại:
{json.dumps(example_2['conversation_segments'], ensure_ascii=False, indent=2)}
Lý do hài hước: {example_2['label']}

### Đoạn hội thoại mới:
Hội thoại:
{json.dumps(conversation['conversation_segments'], ensure_ascii=False, indent=2)}
Lý do hài hước:
"""
    return example_prompt

Gửi prompt cho Together API

In [8]:
# Hàm gửi prompt tới Together API
def analyze_conversation(prompt):
    response = client.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo",
        messages=[{"role": "user", "content": prompt}],
    )
    print( response.usage.total_tokens)
    return response.choices[0].message.content.strip()

Xuất random 2 mẫu làm ví dụ

In [ ]:
# Load dữ liệu
data = load_conversations_and_labels()

# Chọn 2 mẫu bất kỳ làm ví dụ
examples = random.sample(data, 2)
example_1, example_2 = examples

# Phân tích các mẫu còn lại
results = []
for conversation in data:
    if conversation in examples:
        continue  # Bỏ qua 2 mẫu đã chọn
    
    prompt = create_prompt(conversation, example_1, example_2)
    predicted_label = analyze_conversation(prompt)
    print(predicted_label)
    
    results.append({
        "file_name": f"output_{conversation['conversation_id']}.json",
        "predicted_label": predicted_label
    })

Lưu lại kết quả

In [ ]:
# Lưu kết quả vào file CSV
output_file = "D:\\DS310-P11-Nhom15\\results\\meta-llama-Meta-Llama-3.1-405B-Instruct-Turbo.csv"
results_df = pd.DataFrame(results)
results_df.to_csv(output_file, index=False, encoding="utf-8")
print(f"Kết quả đã được lưu vào {output_file}")